In [ ]:
from decodes.core import *
from decodes.io.jupyter_out import JupyterOut
import math

out = JupyterOut.unit_square( )

# Methods of Comparison

Quantifying the possible relationships among pairs of lines or planes is a surprisingly nuanced task. 

We present a detailed discussion of comparison here in order to draw out a latent distinction between the mathematical, programmatic, and visual understandings of these geometries. It turns out that the subtlety of ***drawing comparisons among lines and planes derives not from any mathematical or code property, but from the tension between these formal representations and our expectations of them as visual designers***.

Comparison of LinearEntities fall into two categories: general relations that are valid for any sub-type of LinearEntity (e.g. comparing Rays to Segments), and those relationships that are only valid when considering two of the same sub-type (e.g. comparing a Ray to another Ray).

### Comparing Vectors

The first four of the methods described below faithfully reproduce their mathematical definitions. The last method, `vec.is_similar` is more of a contrivance primarily useful in design applications. Of note for those unfamiliar with these terms is the distinction between equal vectors and coincident vectors.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.06.P17.jpg" style="width: 200px; display: inline;">

<table style="width:600px">
    <tr>
        <th colspan="3" style="text-align:left">*Vector Comparison Methods*</th>
    </tr>
    <tr>
        <th style="width:20%">Equality</th>
        <td style="width:20%">`va==vb`<br>`va.is_equal(vb,tol)`</td>
        <td style="width:80%">True if the two Vecs share the same direction and are of equal length.</td>
    </tr>
    <tr>
        <th style="width:20%">Coincidence</th>
        <td style="width:20%">`va.is_coincident(vb,tol)`</td>
        <td style="width:80%">True if the two Vecs share the same direction.<br>`vec_a` is coincident only to `vec_b`.</td>
    </tr>
    <tr>
        <th style="width:20%">Parallelism</th>
        <td style="width:20%">`va.is_parallel(vb,tol)`</td>
        <td style="width:80%">True if the two Vecs exhibit equal or opposite direction. <br>`vec_a` is parallel to `vec_b` and `vec_c`.</td>
    </tr>
    <tr>
        <th style="width:20%">Perpendicularity</th>
        <td style="width:20%">`va.is_perpendicular(vb,tol)`</td>
        <td style="width:80%">True if the two Vecs are oriented at 90 degrees with respect to one another.<br>`vec_a` is perpendicular to `vec_d`.</td>
    </tr>
    <tr>
        <th style="width:20%">Similarity</th>
        <td style="width:20%">`va.is_similar(vb,tol)`</td>
        <td style="width:80%">True if the two Vecs point in the general same direction, at 90 degrees or less. Useful in determining if inverting one would bring about better alignment.<br>`vec_a` is similar to `vec_b`, `vec_d`, and `vec_e`.</td>
    </tr>
</table>

### Comparing LinearEntities

More constrained comparisons are best implemented at the subclass level, while less constrained ones are appropriate at the superclass level. The table below presents these less constrained comparisons, valid when applied to any one or combination of LinearEntity.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.06.P18.jpg" style="width: 200px; display: inline;">

<table style="width:600px">
    <tr>
        <th colspan="3" style="text-align:left">*LinearEntity Comparison Methods*</th>
    </tr>
    <tr>
        <th style="width:20%">Parallelism</th>
        <td style="width:20%">`la.is_parallel(lb,tol)`</td>
        <td style="width:80%">True if the vectors of the two LinearEntities exhibit equal or opposite direction; Parallelism is a prerequisite for collinearity.<br>`seg_a` is parallel to `seg_b` and `ray_c`.</td>
    </tr>
    <tr>
        <th style="width:20%">Perpendicularity</th>
        <td style="width:20%">`la.is_perpendicular(lb,tol)`</td>
        <td style="width:80%">True if the extension of the two LinearEntities intersect and their vectors are oriented at 90 degrees with respect to one another.<br>`seg_a` is perpendicular to `line_d`.</td>
    </tr>
    <tr>
        <th style="width:20%">Collinearity</th>
        <td style="width:20%">`la.is_collinear(lb,tol_p,tol_a)`</td>
        <td style="width:80%">True if the two LinearEntities lie on the same Line; they are parallel and share a common Point.<br>`seg_a` is collinear only to `seg_b`.</td>
    </tr>
    <tr>
        <th style="width:20%">Point Containment</th>
        <td style="width:20%">`pt in la`<br>`la.contains(pt,tol)`</td>
        <td style="width:80%">True if the given Point lies on the LinearEntity, such that the projected distance is within the given tolerance.</td>
    </tr>
</table>

The effort expended in expressing both lines and planes in terms of vectors is rewarded in the savings gained in implementing comparison methods. Using a vector representation for these entities allows us to construct comparison routines which may often simply call the analogous comparisons of vectors. 

For example, any two LinearEntities are parallel if and only if their vectors are parallel.

In [ ]:
"""
LinearEntity Parallelism
Any two LinearEntities are parallel if their vectors are parallel.
"""
def is_parallel(self,other,tol=None):
    return self.vec.is_parallel(other.vec,tol)

Two Lines are perpendicular if their two vectors are perpendicular and if they
intersect. The perpendicularity of vectors, as we may recall, may be easily checked by $\vec{u_{1}} \bullet \vec{u_{2}} = 0$, but, as we saw in the `is_parallel` method, the code below delegates this functionality to the Vec class.

In [ ]:
"""
LinearEntity Perpendicularity
Any two LinearEntities are perpendicular if their vectors are perpendicular 
and they intersect. We have not yet presented the implementation of 
intersections in Decod.es, but the two relevant lines of code below may be 
self-explanatory.
"""
def is_perpendicular(self,other, tol=None):
    if self.vec.is_perpendicular(other.vec,tol):
        # convert both arguments to Lines
        la, lb = Line(self.spt, self.vec), Line(other.spt, other.vec)
        # determine if lines intersect
        xsec = Intersector()
        if xsec.of(la,lb): return True
    return False

### Comparing Comparing Lines, Rays, and Segments

The particular representation of Lines, Rays, and Segments in code, as a Point and Vec in Decod.es, influences the construction of the relevant methods in ways that feel at odds with the mathematical or visual outlook.

For example, it may make sense in code to consider ***two Segments as equal only if they share precisely the same `seg.spt` and `seg.ept`,*** while from a visual standpoint we might simply consider the commonality of termination Points irrespective of the direction of the Segment. Contrast the `seg.is_coincident()` and `seg.is_equal()` methods in the nearby table.

Similarly, consider that ***in mathematics, infinite lines manifest no particular reference point***. This is at odds with our representation in code, and foregrounds the question of how to characterize the relation between two Lines that are parallel yet are defined by distinct collinear Points. The Decod.es library takes a position on the matter. Whiel the equality of two Lines `line.is_equal()` is sensitive to the location of reference Points and the magnitude of the underlying vectors, their coincidence `line.is_coincident()` is not.

The tables below demonstrate that a host of similar issues arise, each a result of the tension between the mathematical, computational, and visual representations of these elements.

<table style="width:600px">
    <tr>
        <th colspan="3" style="text-align:left">*Line Comparison Methods*</th>
    </tr>
    <tr>
        <th style="width:20%">Equality</th>
        <td style="width:20%">`la == lb`<br>`la.is_equal(lb,tol_p,tol_a)`</td>
        <td style="width:80%">True if the two Lines share a common reference Point and their vectors are coincident.</td>
    </tr>
    <tr>
        <th style="width:20%">Coincidence</th>
        <td style="width:20%">`la.is_coincident(lb,tol_p,tol_a)`</td>
        <td style="width:80%">True if the two Lines share any Point along their length and their vectors are coincident. Collinearity is a similar test requiring only parallel vectors.</td>
    </tr>
</table>


<table style="width:600px">
    <tr>
        <th colspan="3" style="text-align:left">*Ray Comparison Methods*</th>
    </tr>
    <tr>
        <th style="width:20%">Equality / Coincidence</th>
        <td style="width:20%">`ra == rb`<br>`ra.is_equal(rb,tol_p,tol_a)`<br>`ra.is_coincident(rb,tol_p,tol_a)`</td>
        <td style="width:80%">True if the two Rays share a common reference Point and their vectors are coincident.</td>
    </tr>
</table>

<table style="width:600px">
    <tr>
        <th colspan="3" style="text-align:left">*Segment Comparison Methods*</th>
    </tr>
    <tr>
        <th style="width:20%">Equality</th>
        <td style="width:20%">`sa == sb`<br>`sa.is_equal(sb,tol_p,tol_a)`</td>
        <td style="width:80%">True if the two Segments share a common reference Point and their vectors are equal. In this condition, `sa.spt == sb.spt` and `sa.ept == sa.ept`.</td>
    </tr>
    <tr>
        <th style="width:20%">Coincidence</th>
        <td style="width:20%">`sa.is_coincident(sb,tol_p,tol_a)`</td>
        <td style="width:80%">True if the two Segments share some configuration of common termination Points. In this condition, the `spt` of one Segment might match the `ept` of the other Segment. Their vectors may be equal or may be parallel and of equal length.</td>
    </tr>
    <tr>
        <th style="width:20%">Overlap</th>
        <td style="width:20%">`sa.is_overlapping(sb,tol)`</td>
        <td style="width:80%">True if the two Segments are collinear, and share any Point along their length.</td>
    </tr>
    <tr>
        <th style="width:20%">Encompassment</th>
        <td style="width:20%">`sa.is_encompassing(sb,tol)`</td>
        <td style="width:80%">True if the two Segments are collinear, and all the Points along the given Segment are shared by this Segment.</td>
    </tr>
</table>

### Merging Line Segments

Here we examine a particular case in which comparison is useful, and that comes up often in visual design: the merging of line segments.

While the discovery of duplicate lines is often supported in CAD, the merging of visually redundant line segments is typically not. The distinction boils down to the nuanced comparisons demonstrated by the table above. Using those methods, we may define a process for “merging” two Segments that overlap, such that the resulting Segment accurately describes their visual combination.

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.06.P19.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.06.P20.jpg" style="width: 200px; display: inline;">

In [ ]:
"""
Segment Merge
Given two Segments, returns the "maximal" Segment that results from merging the 
two, or False if they do not overlap.
"""
@staticmethod
def merge(seg_a, seg_b):
    # fail if the two given segments don't overlap
    if not seg_a.is_overlapping(seg_b):  return False
    
    lin = seg_a.to_line()
    # collect all the termination Points
    pts = [seg_a.spt,seg_a.ept,seg_b.spt,seg_b.ept]
    # sort termination Points by their projected t-value
    t_vals = sorted( [lin.near(p)[1] for p in pts] )
    # return the longest spanning Segment
    return Segment( lin.eval(t_vals[0]), lin.eval(t_vals[-1]) )

In [ ]:
"""
Segment Mass Merge
Given a List of Segments, merges each with every other so that the minimal 
number of Segments that produce the same visual effect are returned.
"""
def merge_overlapping_segments(segs):
    ret = []
    while len(segs) > 0:
        # pop a segment out of the given list
        seg_a = segs.pop()
        # compare this segment with every other
        for n, seg_b in enumerate(segs):
            merged = Segment.merge(seg_a,seg_b)
            # if a merge succeeds...
            if merged:
                # remove the other segment from the given list
                segs.pop(n)
                # add the merged result to those segments to be checked
                segs.append(merged)
                # flag this segment as having undergone a merge
                seg_a = False
                break
                
        # if no merge flag found, prepare to return this segment
        if seg_a: ret.append(seg_a)
    
    return ret


In [ ]:
"""
Segment Mass Split
"""
def shatter_segments_at_endpoints(segs):
    pts = Point.cull_duplicates([s.spt for s in segs]+[s.ept for s in segs])
    ret = []
    for seg in segs:
        rslts = []
        for pt in pts:
            near = seg.near(pt)
            if near[2] < EPSILON: rslts.append( (near[1],pt) )
        
        rslts.sort()
        for a,b in zip(rslts[:-1],rslts[1:]): ret.append( Segment(a[1],b[1]) )
    return ret
    

### Comparing Planes

The methods needed to compare Planes are very much like the analogous methods for LinearEntity. Since a Plane extends infinitely, they are most analogous to Lines. A similar set of issues arises as a result of the tension between the mathematical, computational, and visual representations of Planes.

For some comparisons, these representations are in harmony. 

For example, mathematically speaking, two planes are parallel if their normal vectors are parallel, and are perpendicular if their normal vectors are perpendicular.

For other comparisons this does not hold true. 

For example, mathematically speaking, two planes are not just parallel, but identical if the equations defining them are identical.

In code, since Planes are defined by an origin Point, we may distinguish between those that share this Point from those that, while coplanar, do not share an origin. A similar distinction may be made between comparisons that test for the coincidence of Plane normals, and those that test only for parallelism.

<table style="width:600px">
    <tr>
        <th colspan="3" style="text-align:left">*Plane Comparison Methods*</th>
    </tr>
    <tr>
        <th style="width:20%">Equality</th>
        <td style="width:20%">`pa == pb`<br>`pa.is_equal(pb,tol_p,tol_a)`</td>
        <td style="width:80%">True if the two Planes share a common reference Point and their vectors are coincident.</td>
    </tr>
    <tr>
        <th style="width:20%">Coincidence</th>
        <td style="width:20%">`pa.is_coincident(pb,tol_p,tol_v)`</td>
        <td style="width:80%">True if the two Planes share any Point in common, and their vectors are coincident.</td>
    </tr>
    <tr>
        <th style="width:20%">Coplanarity</th>
        <td style="width:20%">`pa.is_parallel(pb,tol)`</td> 
        <td style="width:80%">True if the two Planes share any Point in common, and their vectors are parallel.</td>
    </tr>
    <tr>
        <th style="width:20%">Parallelism</th>
        <td style="width:20%">`sa.is_paralell(sb,tol)`</td>
        <td style="width:80%">True if the two Planes share exhibit parallel normal vectors. Note that mathematically speaking, coincident planes are not considered to be parallel.</td>
    </tr>
    <tr>
        <th style="width:20%">Perpendicularity</th>
        <td style="width:20%">`pa.is_perpendicular(pb,tol)`</td>
        <td style="width:80%">True if the normal vectors of the two Planes are oriented at 90 degrees with respect to one another.</td>
    </tr>
</table>

In [ ]:
"""
Plane Parallelism and Perpendicularity
"""
@staticmethod
def is_parallel(pln_a, pln_b):
    return pln_a.vec.is_parallel(pln_b.vec)

@staticmethod
def is_perpendicular(pln_a, pln_b):
    return pln_a.vec.dot(pln_b.vec) == 0
    